In [1]:
#dependencies and load data
import pandas as pd
import hvplot.pandas
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import tree
import pickle

crashes_df = pd.read_csv("Resources\crashes_cleaned_df.csv")
crashes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56414 entries, 0 to 56413
Columns: 159 entries, Unnamed: 0 to STAT_DIV_NAME_Metro
dtypes: float64(7), int64(151), object(1)
memory usage: 68.4+ MB


In [2]:
original_data = pd.read_csv("Resources/Road_Crashes_for_five_Years_Victoria.csv")
original_data = original_data.drop(columns=["X", "Y", "ACCIDENT_NO", "SRNS", "SRNS_ALL", "DIVIDED", "DIVIDED_ALL", "UNKNOWN", "NODE_ID", "OBJECTID"])
original_data = original_data.dropna()

In [3]:
original_data['REGION_NAME'].value_counts()

REGION_NAME
METROPOLITAN SOUTH EAST REGION    20163
METROPOLITAN NORTH WEST REGION    19779
SOUTH WESTERN REGION               4333
EASTERN REGION                     3128
NORTHERN REGION                    3055
NORTH EASTERN REGION               3042
WESTERN REGION                     2914
                                      1
Name: count, dtype: int64

In [4]:
original_data.loc[original_data['REGION_NAME'] == " "]

,ABS_CODE,ACCIDENT_STATUS,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,DAY_OF_WEEK,DCA_CODE,HIT_RUN_FLAG,LIGHT_CONDITION,...,PASSENGERVEHICLE,MOTORCYCLE,PUBLICVEHICLE,DEG_URBAN_NAME,DEG_URBAN_ALL,LGA_NAME_ALL,REGION_NAME_ALL,RMA,RMA_ALL,STAT_DIV_NAME
25344,ABS to receive accident,Finished,22/06/2017,19:30:00+00,Yes,Collision with vehicle,5,REAR END(VEHICLES IN SAME LANE),No,Dark Street lights on,...,2.0,0.0,0.0,MELB_URBAN,MELB_URBAN,MORELAND,METROPOLITAN NORTH WEST REGION,Arterial Other,"Arterial Other,Local Road",Metro


In [5]:
original_data = original_data.drop(original_data.loc[original_data['REGION_NAME'] == " "].index)

In [6]:
model_data_df = crashes_df[['DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5',
                                 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7', 'ACCIDENT_TYPE_Collision with a fixed object',
                                 'ACCIDENT_TYPE_Collision with vehicle', 'ACCIDENT_TYPE_Fall from or in moving vehicle',
                                 'ACCIDENT_TYPE_No collision and no object struck', 'ACCIDENT_TYPE_Other accident',
                                 'ACCIDENT_TYPE_Struck Pedestrian', 'ACCIDENT_TYPE_Struck animal',
                                 'ACCIDENT_TYPE_Vehicle overturned (no collision)',
                                 'ACCIDENT_TYPE_collision with some other object', 'LIGHT_CONDITION_Dark No street lights',
                                 'LIGHT_CONDITION_Dark Street lights off', 'LIGHT_CONDITION_Dark Street lights on',
                                 'LIGHT_CONDITION_Dark Street lights unknown', 'LIGHT_CONDITION_Day',
                                 'LIGHT_CONDITION_Dusk/Dawn', 'LIGHT_CONDITION_Unk.', 'ROAD_GEOMETRY_Cross intersection',
                                 'ROAD_GEOMETRY_Dead end', 'ROAD_GEOMETRY_Multiple intersection',
                                 'ROAD_GEOMETRY_Not at intersection', 'ROAD_GEOMETRY_Private property',
                                 'ROAD_GEOMETRY_Road closure', 'ROAD_GEOMETRY_T intersection', 'ROAD_GEOMETRY_Unknown',
                                 'ROAD_GEOMETRY_Y intersection','SPEED_ZONE_100 km/hr', 'SPEED_ZONE_110 km/hr',
                                 'SPEED_ZONE_40 km/hr', 'SPEED_ZONE_50 km/hr', 'SPEED_ZONE_60 km/hr', 'SPEED_ZONE_70 km/hr',
                                 'SPEED_ZONE_80 km/hr', 'SPEED_ZONE_90 km/hr', 'SPEED_ZONE_Camping grounds or off road',
                                 'SPEED_ZONE_Not known', 'SPEED_ZONE_Other speed limit', 'RMA_ALL_Arterial Highway',
                                 'RMA_ALL_Arterial Highway,Arterial Other', 'RMA_ALL_Arterial Highway,Local Road',
                                 'RMA_ALL_Arterial Other', 'RMA_ALL_Arterial Other,Arterial Highway',
                                 'RMA_ALL_Arterial Other,Local Road', 'RMA_ALL_Freeway', 'RMA_ALL_Freeway,Arterial Other',
                                 'RMA_ALL_Local Road', 'RMA_ALL_Local Road,Arterial Highway',
                                 'RMA_ALL_Local Road,Arterial Other', 'RMA_ALL_Other', 'SEVERITY_Fatal accident',
                                 'SEVERITY_Non injury accident', 'SEVERITY_Other injury accident',
                                 'SEVERITY_Serious injury accident', 'REGION_NAME_EASTERN REGION',
                                 'REGION_NAME_METROPOLITAN NORTH WEST REGION', 'REGION_NAME_METROPOLITAN SOUTH EAST REGION',
                                 'REGION_NAME_NORTH EASTERN REGION', 'REGION_NAME_NORTHERN REGION', 
                                 'REGION_NAME_SOUTH WESTERN REGION', 'REGION_NAME_WESTERN REGION']]
model_data_df.head()

,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7,ACCIDENT_TYPE_Collision with a fixed object,ACCIDENT_TYPE_Collision with vehicle,ACCIDENT_TYPE_Fall from or in moving vehicle,...,SEVERITY_Non injury accident,SEVERITY_Other injury accident,SEVERITY_Serious injury accident,REGION_NAME_EASTERN REGION,REGION_NAME_METROPOLITAN NORTH WEST REGION,REGION_NAME_METROPOLITAN SOUTH EAST REGION,REGION_NAME_NORTH EASTERN REGION,REGION_NAME_NORTHERN REGION,REGION_NAME_SOUTH WESTERN REGION,REGION_NAME_WESTERN REGION
0,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0


In [7]:
column_list = model_data_df.columns

In [8]:
scaler = StandardScaler().fit(model_data_df)

X_scaled = scaler.transform(model_data_df)

In [9]:
y = original_data['DEG_URBAN_NAME']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=78)

In [11]:
treem = tree.DecisionTreeClassifier()
treem = treem.fit(X_train, y_train.ravel())
treem_predictions = treem.predict(X_test)

In [12]:
print(classification_report(y_test, treem_predictions))

                         precision    recall  f1-score   support

LARGE_PROVINCIAL_CITIES       0.60      0.67      0.63       796
          MELBOURNE_CBD       0.33      0.27      0.30       173
             MELB_URBAN       0.92      0.96      0.94      8755
         RURAL_VICTORIA       0.83      0.79      0.81      3022
           SMALL_CITIES       0.46      0.37      0.41       790
            SMALL_TOWNS       0.05      0.03      0.04       137
                  TOWNS       0.23      0.18      0.20       431

               accuracy                           0.83     14104
              macro avg       0.49      0.47      0.47     14104
           weighted avg       0.82      0.83      0.82     14104



In [15]:
pickle.dump(treem, open('tree_model.sav', 'wb'))

In [16]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

In [13]:
treem_predictions

array(['MELB_URBAN', 'RURAL_VICTORIA', 'MELB_URBAN', ..., 'MELB_URBAN',
       'MELB_URBAN', 'RURAL_VICTORIA'], dtype=object)

In [53]:
LGA_to_replace = ["MILDURA",
"MACEDON RANGES",
"MURRINDINDI",
"COLAC OTWAY",
"MOORABOOL",
"SOUTH GIPPSLAND",
"SURF COAST",
"CAMPASPE",
"BASS COAST",
"MOIRA",
"WANGARATTA",
"GOLDEN PLAINS",
"WARRNAMBOOL",
"WODONGA",
"CORANGAMITE",
"MANSFIELD",
"MOYNE",
"ALPINE",
"HEPBURN",
"GLENELG",
"STRATHBOGIE",
"SWAN HILL",
"HORSHAM",
"INDIGO",
"BENALLA",
"NORTHERN GRAMPIANS",
"MOUNT ALEXANDER",
"SOUTHERN GRAMPIANS",
"TOWONG",
"PYRENEES",
"ARARAT",
"CENTRAL GOLDFIELDS",
"LODDON",
"GANNAWARRA",
"BULOKE",
"HINDMARSH",
"WEST WIMMERA",
"YARRIAMBIACK",
"(MOUNT HOTHAM)",
"QUEENSCLIFFE",
"(LAKE MOUNTAIN)",
"(MOUNT BULLER)",
"(FALLS CREEK)",
"(MOUNT BAW BAW)",
"(FRENCH ISLAND)",
" ",
"(MOUNT STIRLING)"]

for lga in LGA_to_replace:
    original_data['LGA_NAME'] = original_data['LGA_NAME'].replace(lga,"Other")

# Check to make sure binning was successful
original_data['LGA_NAME'].value_counts()

LGA_NAME
Other                   8383
MELBOURNE               3049
CASEY                   2420
GEELONG                 2273
DANDENONG               1936
HUME                    1870
BRIMBANK                1683
MONASH                  1663
WHITTLESEA              1656
MORELAND                1619
YARRA RANGES            1582
YARRA                   1395
DAREBIN                 1377
KINGSTON                1355
WYNDHAM                 1270
BOROONDARA              1231
WHITEHORSE              1208
MORNINGTON PENINSULA    1163
STONNINGTON             1162
KNOX                    1134
PORT PHILLIP            1103
GLEN EIRA               1083
BENDIGO                 1077
BALLARAT                1040
FRANKSTON               1004
MELTON                   955
MOONEE VALLEY            905
CARDINIA                 882
MAROONDAH                851
BANYULE                  841
MARIBYRNONG              792
HOBSONS BAY              770
MANNINGHAM               732
SHEPPARTON               730
BAYSI

In [54]:
layer_2_data = crashes_df[['DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5',
                                 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7', 'ACCIDENT_TYPE_Collision with a fixed object',
                                 'ACCIDENT_TYPE_Collision with vehicle', 'ACCIDENT_TYPE_Fall from or in moving vehicle',
                                 'ACCIDENT_TYPE_No collision and no object struck', 'ACCIDENT_TYPE_Other accident',
                                 'ACCIDENT_TYPE_Struck Pedestrian', 'ACCIDENT_TYPE_Struck animal',
                                 'ACCIDENT_TYPE_Vehicle overturned (no collision)',
                                 'ACCIDENT_TYPE_collision with some other object', 'LIGHT_CONDITION_Dark No street lights',
                                 'LIGHT_CONDITION_Dark Street lights off', 'LIGHT_CONDITION_Dark Street lights on',
                                 'LIGHT_CONDITION_Dark Street lights unknown', 'LIGHT_CONDITION_Day',
                                 'LIGHT_CONDITION_Dusk/Dawn', 'LIGHT_CONDITION_Unk.', 'ROAD_GEOMETRY_Cross intersection',
                                 'ROAD_GEOMETRY_Dead end', 'ROAD_GEOMETRY_Multiple intersection',
                                 'ROAD_GEOMETRY_Not at intersection', 'ROAD_GEOMETRY_Private property',
                                 'ROAD_GEOMETRY_Road closure', 'ROAD_GEOMETRY_T intersection', 'ROAD_GEOMETRY_Unknown',
                                 'ROAD_GEOMETRY_Y intersection','SPEED_ZONE_100 km/hr', 'SPEED_ZONE_110 km/hr',
                                 'SPEED_ZONE_40 km/hr', 'SPEED_ZONE_50 km/hr', 'SPEED_ZONE_60 km/hr', 'SPEED_ZONE_70 km/hr',
                                 'SPEED_ZONE_80 km/hr', 'SPEED_ZONE_90 km/hr', 'SPEED_ZONE_Camping grounds or off road',
                                 'SPEED_ZONE_Not known', 'SPEED_ZONE_Other speed limit', 'TOTAL_PERSONS', 'INJ_OR_FATAL',
                                 'MALES', 'FEMALES', 'UNLICENCSED', 'RMA_Arterial Highway', 'RMA_Arterial Other',
                                 'RMA_Freeway', 'RMA_Local Road', 'RMA_Non Arterial', 'RMA_ALL_Arterial Highway',
                                 'RMA_ALL_Arterial Highway,Arterial Other', 'RMA_ALL_Arterial Highway,Local Road',
                                 'RMA_ALL_Arterial Other', 'RMA_ALL_Arterial Other,Arterial Highway',
                                 'RMA_ALL_Arterial Other,Local Road', 'RMA_ALL_Freeway', 'RMA_ALL_Freeway,Arterial Other',
                                 'RMA_ALL_Local Road', 'RMA_ALL_Local Road,Arterial Highway',
                                 'RMA_ALL_Local Road,Arterial Other', 'RMA_ALL_Other', 'SEVERITY_Fatal accident',
                                 'SEVERITY_Non injury accident', 'SEVERITY_Other injury accident',
                                 'SEVERITY_Serious injury accident', 'REGION_NAME_EASTERN REGION',
                                 'REGION_NAME_METROPOLITAN NORTH WEST REGION', 'REGION_NAME_METROPOLITAN SOUTH EAST REGION',
                                 'REGION_NAME_NORTH EASTERN REGION', 'REGION_NAME_NORTHERN REGION', 
                                 'REGION_NAME_SOUTH WESTERN REGION', 'REGION_NAME_WESTERN REGION', 
                                 'STAT_DIV_NAME_Country', 'STAT_DIV_NAME_Metro', 'DEG_URBAN_NAME_LARGE_PROVINCIAL_CITIES', 
                                 'DEG_URBAN_NAME_MELBOURNE_CBD', 'DEG_URBAN_NAME_MELB_URBAN', 'DEG_URBAN_NAME_RURAL_VICTORIA', 
                                 'DEG_URBAN_NAME_SMALL_CITIES', 'DEG_URBAN_NAME_SMALL_TOWNS', 'DEG_URBAN_NAME_TOWNS']]
layer_2_data.head()

,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7,ACCIDENT_TYPE_Collision with a fixed object,ACCIDENT_TYPE_Collision with vehicle,ACCIDENT_TYPE_Fall from or in moving vehicle,...,REGION_NAME_WESTERN REGION,STAT_DIV_NAME_Country,STAT_DIV_NAME_Metro,DEG_URBAN_NAME_LARGE_PROVINCIAL_CITIES,DEG_URBAN_NAME_MELBOURNE_CBD,DEG_URBAN_NAME_MELB_URBAN,DEG_URBAN_NAME_RURAL_VICTORIA,DEG_URBAN_NAME_SMALL_CITIES,DEG_URBAN_NAME_SMALL_TOWNS,DEG_URBAN_NAME_TOWNS
0,0,0,0,1,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
1,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [55]:
scaler_2 = StandardScaler().fit(layer_2_data)

X_scaled_2 = scaler_2.transform(layer_2_data)

In [56]:
y_2 = original_data['LGA_NAME']

In [57]:
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_scaled_2, y_2, random_state=78)

In [58]:
treem_2 = tree.DecisionTreeClassifier()
treem_2 = treem_2.fit(X_2_train, y_2_train.ravel())
treem_2_predictions = treem_2.predict(X_2_test)

In [59]:
print(classification_report(y_2_test, treem_2_predictions))

                      precision    recall  f1-score   support

            BALLARAT       0.84      0.92      0.88       247
             BANYULE       0.13      0.14      0.13       236
             BAW BAW       0.29      0.30      0.30       164
             BAYSIDE       0.07      0.12      0.09       170
             BENDIGO       0.69      0.78      0.73       249
          BOROONDARA       0.14      0.18      0.15       303
            BRIMBANK       0.14      0.17      0.15       428
            CARDINIA       0.45      0.51      0.48       203
               CASEY       0.23      0.25      0.24       606
           DANDENONG       0.16      0.18      0.17       492
             DAREBIN       0.15      0.15      0.15       359
      EAST GIPPSLAND       0.28      0.33      0.31       141
           FRANKSTON       0.09      0.08      0.08       251
             GEELONG       0.82      0.86      0.84       555
           GLEN EIRA       0.08      0.07      0.07       281
       